In [1]:
import os
import time
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import time

In [2]:
df_train = pd.read_csv('train_lemmatized_sw.csv')

In [3]:
df_train.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'PRODUCT_ID', 'TITLE', 'BULLET_POINTS',
       'DESCRIPTION', 'PRODUCT_TYPE_ID', 'PRODUCT_LENGTH', 'T_BP', 'T_BP_D',
       'T_D', 'F1'],
      dtype='object')

In [4]:
del df_train['Unnamed: 0']
del df_train['Unnamed: 0.1']
del df_train['F1']

In [5]:
df_test = pd.read_csv('test_processed.csv')

In [6]:
df_test.columns

Index(['Unnamed: 0', 'PRODUCT_ID', 'TITLE', 'BULLET_POINTS', 'DESCRIPTION',
       'PRODUCT_TYPE_ID', 'T_BP', 'T_BP_D', 'T_D'],
      dtype='object')

In [7]:
del df_test['Unnamed: 0']

In [8]:
df_train["T_BP_D"].replace(np.nan, "", inplace=True)

In [9]:
df_train['T_BP_D'].isna().sum()


0

In [10]:
import pandas as pd
from sklearn.utils import resample

# Load the dataset

# Separate the column with 19k entries
col = df_train['T_BP_D']

# Sample 50% of the column while preserving the distribution
sampled_col = resample(col, n_samples=0.01*len(col), random_state=25, stratify=col)

# Get the indices of the sampled entries
indices = sampled_col.index

# Sample the original dataframe based on the indices
sampled_df = df_train.loc[indices]

In [11]:
df_train = sampled_df.copy()

In [13]:
df_train.shape

(22496, 9)

In [14]:
y = df_train['PRODUCT_LENGTH']

In [15]:
df_test.shape

(734736, 8)

In [16]:
df_train['T_BP_D'].isna().sum()


0

In [17]:
X_train1 = df_train.copy()

In [18]:
X_train1.shape, df_test.shape

((22496, 9), (734736, 8))

In [19]:
X_test1 = df_test.copy()

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an instance of TfidfVectorizer

s = time.time()
vectorizer = TfidfVectorizer(lowercase = True, max_features=20000, stop_words='english', ngram_range=(1,4))

X_train1_tbpd_tfidf = vectorizer.fit_transform(X_train1['T_BP_D'])
# X_test1_tbpd_tfidf = vectorizer.transform(X_test1['T_BP_D'])


e = time.time()

print(str((e-s)/60) + " mins")


0.13240677913029988 mins


In [21]:
X_train1_tbpd_tfidf = X_train1_tbpd_tfidf.toarray()

In [22]:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

In [23]:
s = time.time()

pca = PCA(n_components = 5000)
X_pca_tbpd = pca.fit_transform(X_train1_tbpd_tfidf)
# X_pca_tbpd_test = pca.transform(X_test1_tbpd_tfidf)


# Print the explained variance ratio for normal PCA
print("Explained Variance Ratio (Normal PCA): ", pca.explained_variance_ratio_)
e = time.time()
print(str((e-s)/60) + " mins")

Explained Variance Ratio (Normal PCA):  [3.26063187e-02 7.86715946e-03 7.00653403e-03 ... 1.05473748e-05
 1.05338943e-05 1.04999483e-05]
5.354553016026815 mins


In [24]:
sum(pca.explained_variance_ratio_)

0.9855253102232971

In [25]:
X_pca_tbpd = pd.DataFrame(X_pca_tbpd)

In [26]:
product_ids = X_train1['PRODUCT_TYPE_ID']

In [27]:
product_ids = pd.DataFrame(product_ids)

In [28]:
final_df = pd.DataFrame(X_pca_tbpd)

In [29]:
pro_df = pd.DataFrame(product_ids)

In [30]:
final_df.shape, pro_df.shape

((22496, 5000), (22496, 1))

In [31]:
pro_df = pro_df.reset_index()

In [32]:
df= pd.concat([final_df, pro_df], axis = 1)

In [33]:
del df['index']

In [34]:
df.shape

(22496, 5001)

In [35]:
df

,0,1,2,3,4,5,6,7,8,9,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,PRODUCT_TYPE_ID
0,-0.043986,0.051683,-0.008200,-0.051956,-0.014786,-0.000243,-0.001790,-0.001364,0.029117,-0.014011,...,-0.007031,-0.004286,-0.006974,-0.000975,0.013666,0.004034,0.005914,-0.001805,0.012241,5994
1,-0.045049,-0.033840,-0.008355,0.017111,0.015382,-0.006532,-0.012247,-0.012043,-0.006017,-0.004784,...,-0.000087,-0.001447,-0.000091,-0.000304,-0.000094,-0.002118,-0.000345,0.000612,-0.002070,804
2,-0.036866,0.049976,-0.004286,-0.001697,0.001895,-0.003446,-0.000067,-0.011159,0.024914,-0.025028,...,-0.002069,0.000926,-0.000549,-0.002351,0.000141,-0.002734,0.000260,-0.001680,-0.000574,1171
3,-0.047037,0.066503,-0.031362,-0.059729,-0.008454,0.005194,-0.007336,0.005672,-0.200506,0.116161,...,0.000806,-0.003967,0.003197,-0.000829,0.007040,0.006401,0.006144,0.002372,0.013057,3246
4,-0.045081,-0.058556,0.003661,0.002485,-0.000257,-0.011659,-0.004318,-0.020943,-0.008545,-0.004072,...,0.000352,0.000239,0.000008,0.000055,-0.000181,0.000282,-0.000277,-0.000059,0.000134,811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22491,-0.047452,-0.069034,0.003434,0.001012,-0.001112,-0.017312,-0.008214,-0.028033,-0.004921,0.000813,...,0.000113,-0.000092,-0.000482,0.001032,-0.000234,0.001017,0.000410,0.000051,-0.000445,6104
22492,-0.055282,0.218711,0.687797,0.210664,-0.021048,0.018539,-0.017403,0.024334,-0.032950,0.038129,...,-0.000163,0.000734,0.000177,0.000631,-0.003204,0.000184,0.000737,0.001838,-0.002770,1634
22493,0.746341,-0.008986,0.021434,-0.046357,-0.066863,-0.006858,0.015374,0.006129,0.010973,-0.001694,...,0.000332,0.000026,0.000070,0.000301,-0.000163,0.000508,0.000022,0.000022,-0.000122,12064
22494,-0.053639,0.172922,0.491914,0.141445,-0.016435,0.013524,-0.013128,0.009651,0.002744,0.002430,...,-0.001021,0.004640,-0.000209,0.004398,-0.012664,0.003449,-0.001069,0.003691,-0.005105,1634


In [68]:
from sklearn.neighbors import KNeighborsRegressor

# create a KNN regression model with k = 5
knn = KNeighborsRegressor(n_neighbors=5, n_jobs = -1)

# train the model on the training data
knn.fit(df, y)

# use the model to make predictions on the test data
# y_pred = knn.predict(X_test)


KNeighborsRegressor(n_jobs=-1)

In [37]:
df_test["T_BP_D"].replace(np.nan, "", inplace = True)


In [38]:
X_test_tbpd_tfidf = vectorizer.transform(df_test['T_BP_D'])

In [40]:
X_test_tbpd_tfidf = X_test_tbpd_tfidf.toarray()

In [41]:
s = time.time()

X_pca_tbpd_test = pca.transform(X_test_tbpd_tfidf)
e = time.time()
print(str((e-s)/60), "mins")

In [42]:
X_pca_tbpd_test = pd.DataFrame(X_pca_tbpd_test)

In [43]:
pro_test = df_test["PRODUCT_TYPE_ID"]

In [44]:
final_testing = pd.concat([pro_test, X_pca_tbpd_test], axis=1)

In [45]:
final_testing.shape

(734736, 5001)

In [69]:
s = time.time()

predicted = knn.predict(final_testing)

e = time.time()

print(str((e-s)/60), "mins")

26.875272619724274 mins


In [71]:
pro_test = df_test["PRODUCT_TYPE_ID"]

In [72]:
predicted = pd.DataFrame(predicted)

In [73]:
predicted.columns = ["PRODUCT_LENGTH"]

In [74]:
len(df_test["PRODUCT_ID"])

734736

In [75]:
pro_test = pd.DataFrame(pro_test)

In [76]:
outputs = {'PRODUCT_ID': list(df_test["PRODUCT_ID"]), 'PRODUCT_LENGTH': list(predicted["PRODUCT_LENGTH"])}

len(outputs)

2

In [77]:
outputs = pd.DataFrame(outputs)

In [78]:
outputs

,PRODUCT_ID,PRODUCT_LENGTH
0,604373,531.990551
1,1729783,532.390551
2,1871949,531.990551
3,1107571,531.990551
4,624253,531.990551
...,...,...
734731,921419,531.990551
734732,2456362,565.200000
734733,841529,531.990551
734734,1190194,531.990551


In [79]:
outputs.to_csv("knn_m3_5nbrs.csv")